Yi Tang & Chenxi Yang 

# DATA 586 Project 2

### Introduction

In this mini project 2, we work on forecasting the unit sales of various products sold in the USA by Walmart as precisely as possible using RNN model, one of the effective models in machine learning, to improve forecast accuracy.

All data are retrieved from https://www.kaggle.com/c/m5-forecasting-accuracy/data, which covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. 

The data comprises 3049 individual products from 3 categories and 7 departments, sold in 10 stores in 3 states from `d1` to `d1913`, which are 1913 days in total. The hierachical aggregation captures the combinations of these factors.

Our goal of this project is to apply this robust dataset to forecast daily sales for the next 28 days (from `d1914` to `d1941`) and improve forecasting accuracy.

We aim to advance the theory and practice of forecasting. The methods used can be applied in various business areas, such as setting up appropriate inventory or service levels. Through its business support and training, the MOFC will help distribute the tools and knowledge so others can achieve more accurate and better calibrated forecasts, reduce waste and be able to appreciate uncertainty and its risk implications.

*Note: this .ipynb runs at jupyter notebook.*

### Import packages 

Import all packages used in this project and set up for the operation path at first.


*Note: GPU acceleration is not used for the training in this project.*

In [0]:
# Import packages
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
os.chdir("/Users/ty/Desktop/DATA 586/project/project2/")

### Perpare Data

Our objective for this project is to predict item sales at stores in various locations for two 28-day time periods. 

According to the M5 Participants Guide, inside the entire dataset `m5-forecasting-accuracy`, it can be observed that there are 4 separate csv files, including `calender.csv`, `sales_train_validation.csv`, `sell_prices.csv` and `sample_submission.csv`.

We want to go through all data sets and prepare them for modelling.

#### Load Data

Load all data sets one by one.

***calender.csv*** contains information about the dates on which the products are sold.

The calender data is given for all the 1913 days in the sales data (actually we have 1969 days.
There are maximum 2 events in a day for which the event names and the event types are given.

Dates together with related features like day-of-the week, month, year, and an 3 binary flags for whether the stores in each state allowed purchases with SNAP food stamps at this date (1) or not (0).

*Note: The calendar data has 1969 rows and 14 columns.*

In [0]:
calendar = pd.read_csv("m5-forecasting-accuracy/calendar.csv")
calendar.head(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0


In [0]:
calendar.shape

(1969, 14)

***sell_prices.csv*** contains information about the price of the products sold per store and date.

For each item sold in a store located in California, Texas and Winscoin, the price of it is given with the exact time of purchase.

The store and item IDs together with the sales price of the item as a weekly average.

*Note: The sell_prices data has 6841121 rows and 4 columns.*

In [0]:
sell_prices = pd.read_csv("m5-forecasting-accuracy/sell_prices.csv")
sell_prices.head(2)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58


In [0]:
sell_prices.shape

(6841121, 4)

***sales_train_evaluation.csv*** is available once month before competition deadline. It will include sales from `d1` to `d1941`.

It has 1 column for each of the 1941 days from 2011-01-29 and 2016-05-22; not including the validation period of 28 days until 2016-06-19. The columns d_1 to d_1913 leads to the sales of the given item in that store on the nth day for 1913 days.It also includes the IDs for item, department, category, store, and state.

The unique identifier is a concatenation of item_id and store_id. As observed, there are 3049 unique items and 10 unique stores, which result in the total 30490 rows.

*Note: The sales_train_evaluation data has 30490 rows and 1919 columns.*

In [0]:
sell_train_validation = pd.read_csv("m5-forecasting-accuracy/sales_train_validation.csv")
sell_train_validation.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [0]:
sell_train_validation.shape

(30490, 1919)

***sample_submission.csv*** includes the correct format for submissions, which is a template for the final submission file.

*Note: The sample_submission data has 60980 rows and 29 columns.*

In [0]:
sample_submission = pd.read_csv("m5-forecasting-accuracy/sample_submission.csv")
sample_submission.head(2)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
sample_submission.shape

(60980, 29)

#### Clean Data

As our dataset is quite large, so we apply a helper function for reducing memory usage.

In [0]:
# Function for reducing memory usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

Another helper function is applied to transfer strings to numbers.

In [0]:
# Function for transferring strings to numbers

def trans_to_num(columnname, dataf):
    temps = dataf[columnname].unique()
    df = pd.DataFrame()
    df[columnname] = temps
    df = df.sort_values(by=[columnname])
    df["num"] = range(len(temps))
    dataf = dataf.merge(df, how="left", on=columnname)
    dataf[columnname] = dataf["num"]
    dataf = dataf.drop(columns=["num"])
    del temps,df
    
    return dataf

For `calendar`, columns `date` and `weekday` are dropped. In addition, transfer strings to numbers on columns `event_name_1`, `event_type_1`, `event_name_2` and `event_type_2`.

*Note: 70.8% of memory usage has been decreased.*

In [0]:
# calendar

calendar_new = calendar.drop(columns=["date", "weekday"])
calendar_new = trans_to_num("event_name_1", calendar_new)
calendar_new = trans_to_num("event_type_1", calendar_new)
calendar_new = trans_to_num("event_name_2", calendar_new)
calendar_new = trans_to_num("event_type_2", calendar_new)
calendar = calendar_new
del calendar_new

In [0]:
# calendar

temps1 = calendar[calendar.columns[0:9]]
temps2 = pd.DataFrame(np.repeat(temps1.values, len(calendar.columns[9:]), axis=0))
temps2.columns = temps1.columns

# stack snap-columns

temps3 = calendar[calendar.columns[9:]]
temps3 = np.array(temps3.stack())

# concate together

temps4 = pd.DataFrame()
temps4["state_id"] = np.array(["CA", "TX", "WI"])
temps4 = pd.concat([temps4]*len(calendar), ignore_index=True)
temps5 = pd.concat([temps2, pd.Series(temps4["state_id"], name="state_id"), pd.Series(temps3, name="snap")], axis=1)
calendar = temps5
calendar[calendar.columns[[0,1,2,3,5,6,7,8,10]]]=calendar[calendar.columns[[0,1,2,3,5,6,7,8,10]]].apply(pd.to_numeric)
calendar = trans_to_num("state_id", calendar)
calendar = reduce_mem_usage(calendar)
del temps1, temps2, temps3, temps4, temps5

calendar["d_n"] = calendar["d"].map(lambda x: int(x[2:]))
calendar = calendar.drop(columns=["d"])
calendar = calendar.sort_values(by=["d_n", "state_id"])
calendar = calendar[calendar.d_n<=1913+28]
calendar.head(2)

Memory usage after optimization is: 0.16 MB
Decreased by 70.8%


,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,state_id,snap,d_n
0,11101,1,1,2011,30,4,4,2,0,0,1
1,11101,1,1,2011,30,4,4,2,1,0,1


For `sell_prices`, a new column `id` is added, which combines `item_id` and `store_id`.

*Note: 30.0% of memory usage has been decreased.*

In [0]:
# sell_prices

sell_prices["id"] = sell_prices["item_id"] + "_" + sell_prices["store_id"] + "_validation"
sell_prices = reduce_mem_usage(sell_prices)
sell_prices.head(2)

Memory usage after optimization is: 182.68 MB
Decreased by 30.0%


,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.578125,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.578125,HOBBIES_1_001_CA_1_validation


For `sell_train_validation`, firstly, transfer strings to numbers on columns `item_id`, `dept_id`, `cat_id`, `store_id` and `state_id` ,and drop the column `id`.

Then, `day-sale-columns` is stacked and a new column `d_n` is created to represent the number of days. 

At last, all separate parts are concated together.

*Note: 61.5% of memory usage has been decreased.*

In [0]:
# sell_train_validation

temps1 = sell_train_validation[sell_train_validation.columns[0:6]]
temps1 = trans_to_num("item_id", temps1)
temps1 = trans_to_num("dept_id", temps1)
temps1 = trans_to_num("cat_id", temps1)
temps1 = trans_to_num("store_id", temps1)
temps1 = trans_to_num("state_id", temps1)
temps1 = temps1.drop(columns=["id"]).copy()
temps1 = reduce_mem_usage(temps1)
temps2 = pd.DataFrame(np.repeat(temps1.values, len(sell_train_validation.columns[6:]), axis=0))
temps2.columns = temps1.columns

# stack day-sale-columns

temps3 = sell_train_validation[sell_train_validation.columns[6:]]
temps3 = np.array(temps3.stack())

# concate together

temps4 = pd.DataFrame()
temps4["d"] = np.array(sell_train_validation.columns[6:])
temps4["d_n"] = temps4["d"].map(lambda x: int(x[2:]))
temps4 = temps4.drop(columns=["d"]).copy()
temps4 = pd.concat([temps4]*len(sell_train_validation), ignore_index=True)
sales = pd.concat([temps2, temps4, pd.Series(temps3,name="sales")], axis=1)
sales = reduce_mem_usage(sales)
del temps1, temps2, temps3, temps4, sell_train_validation

sales.head(2)

Memory usage after optimization is: 0.41 MB
Decreased by 70.8%
Memory usage after optimization is: 556.25 MB
Decreased by 61.5%


,item_id,dept_id,cat_id,store_id,state_id,d_n,sales
0,1437,3,1,0,0,1,0
1,1437,3,1,0,0,2,0


For `sample_submission`, firstly, the columns `item_id`, `dept_id`, `cat_id`, `store_id` and `state_id` are slightly modified on length and transferred from strings to numbers.

Then, `day-sale-columns` is stacked.
At last, all separate parts are concated together.

*Note: 65.4% of memory usage has been decreased.*

In [0]:
sample_submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
# sample_submission

sample_submission = sample_submission[sample_submission["id"].str.contains("validation")]
sample_submission["item_id"] = sample_submission.id.map(lambda x: x[:-16])
sample_submission["dept_id"] = sample_submission.id.map(lambda x: x[:-20])
sample_submission["cat_id"] = sample_submission.id.map(lambda x: x[:-22])
sample_submission["store_id"] = sample_submission.id.map(lambda x: x[-15:-11])
sample_submission["state_id"] = sample_submission.id.map(lambda x: x[-15:-13])

temps1 = sample_submission[sample_submission.columns[[0,29,30,31,32,33]]]
temps1 = trans_to_num("item_id",temps1)
temps1 = trans_to_num("dept_id",temps1)
temps1 = trans_to_num("cat_id",temps1)
temps1 = trans_to_num("store_id",temps1)
temps1 = trans_to_num("state_id",temps1)
temps1 = reduce_mem_usage(temps1)

sample_id = temps1.copy() 
temps1 = temps1.drop(columns=["id"]).copy()
temps2 = pd.DataFrame(np.repeat(temps1.values, 28, axis=0))
temps2.columns = temps1.columns

# stack day-sale-columns

temps3 = sample_submission[sample_submission.columns[1:29]]
temps3 = np.array(temps3.stack())

# concate together

temps4 = pd.DataFrame()
temps4["d_n"] = np.array(range(1914, 1942))
temps4 = pd.concat([temps4]*len(sample_submission), ignore_index=True)
sample = pd.concat([temps2, temps4, pd.Series(temps3, name="sales")], axis=1)
sample_submission = reduce_mem_usage(sample)
del temps1, temps2, temps3, temps4, sample

sample_submission.head(2)

Memory usage after optimization is: 0.64 MB
Decreased by 60.7%
Memory usage after optimization is: 7.33 MB
Decreased by 65.4%


,item_id,dept_id,cat_id,store_id,state_id,d_n,sales
0,1437,3,1,0,0,1914,0
1,1437,3,1,0,0,1915,0


*Note: **sample_id** is a dataframe that `hold id`, `item_id`, `dept_id`, `cat_id`, `store_id`, and `state_id`.*

In [0]:
sample_id.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0


#### Merge data

In order to progress the further prediction procedure, we construct a dataframe named `sales`, which is a dataframe that combines another three dataframes (`calandar`, `sell_prices`, `sell_train_validation`).

In [0]:
# sample_submission merge calendar

sample = sample_submission.copy()
sample = pd.merge(sample, calendar, how="left", left_on=["d_n", "state_id"], right_on = ["d_n", "state_id"])
sample = trans_to_num("state_id", sample)
sample_submission = reduce_mem_usage(sample)
del sample

sample_submission.head(2)                    

Memory usage after optimization is: 22.80 MB
Decreased by 20.0%


,item_id,dept_id,cat_id,store_id,state_id,d_n,sales,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap
0,1437,3,1,0,0,1914,0,11613,3,4,2016,30,4,4,2,0
1,1437,3,1,0,0,1915,0,11613,4,4,2016,30,4,4,2,0


In [0]:
# sales merge calendar

sales_new = sales.copy()
sales_new = pd.merge(sales_new, calendar, how="left", left_on=["d_n", "state_id"], right_on = ["d_n", "state_id"])
del calendar

sales_new = trans_to_num("state_id", sales_new)
sales = reduce_mem_usage(sales_new)
del sales_new

sales.head(2)

Memory usage after optimization is: 1613.13 MB
Decreased by 19.4%


,item_id,dept_id,cat_id,store_id,state_id,d_n,sales,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap
0,1437,3,1,0,0,1,0,11101,1,1,2011,30,4,4,2,0
1,1437,3,1,0,0,2,0,11101,2,1,2011,30,4,4,2,0


In [0]:
# sample_submission merge sell_prices

sell_prices_short = pd.merge(sell_prices, sample_id, how="left", left_on=["id"], right_on = ["id"])
sell_prices_short = sell_prices_short.select_dtypes(exclude='object')
sell_prices_short.columns = ["wm_yr_wk", "sell_price", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
del sell_prices

sample = pd.merge(sample_submission, sell_prices_short, how="left", left_on=["dept_id", "item_id", "wm_yr_wk", "cat_id", "store_id", "state_id"], right_on=["dept_id", "item_id", "wm_yr_wk", "cat_id", "store_id", "state_id"])
sample_submission = reduce_mem_usage(sample)
del sample

sample_submission.head(2)

Memory usage after optimization is: 24.43 MB
Decreased by 0.0%


,item_id,dept_id,cat_id,store_id,state_id,d_n,sales,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price
0,1437,3,1,0,0,1914,0,11613,3,4,2016,30,4,4,2,0,8.382812
1,1437,3,1,0,0,1915,0,11613,4,4,2016,30,4,4,2,0,8.382812


In [0]:
# sales merge sell_prices

sales_new = pd.merge(sales, sell_prices_short, how="left", left_on=["dept_id", "item_id", "wm_yr_wk", "cat_id", "store_id", "state_id"], right_on=["dept_id", "item_id", "wm_yr_wk", "cat_id", "store_id", "state_id"])
del sell_prices_short

sales_new["sell_price"] = sales_new["sell_price"].fillna(0) 
sales = reduce_mem_usage(sales_new)
del sales_new

sales.head(2)

Memory usage after optimization is: 1724.38 MB
Decreased by 0.0%


,item_id,dept_id,cat_id,store_id,state_id,d_n,sales,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price
0,1437,3,1,0,0,1,0,11101,1,1,2011,30,4,4,2,0,0.0
1,1437,3,1,0,0,2,0,11101,2,1,2011,30,4,4,2,0,0.0


In [0]:
sales.shape

(58327370, 17)

For our final dataframe `sales`, there are 58327370 rows and 17 columns.

#### Concat data

At first, concat `sales` and `sample_submission` to form a dataset `alldata` that contains all data.

In [0]:
alldata = pd.concat([sales, sample_submission], sort=False)
alldata = reduce_mem_usage(alldata)
del sales, sample_submission

Memory usage after optimization is: 1749.62 MB
Decreased by 0.0%


The following step is to merge `alldata` with `sample_id` on `state_id`, `store_id`, `cat_id`, `dept_id` and `item_id`

All values are sorted by `id` and `d_n` and all indexs are reset.

In [0]:
alldata_full = pd.merge(alldata, sample_id, how="left",
                     left_on=["state_id", "store_id", "cat_id", "dept_id", "item_id"],
                     right_on=["state_id", "store_id", "cat_id", "dept_id", "item_id"])
alldata_full = reduce_mem_usage(alldata_full)
del alldata

alldata_full = alldata_full.sort_values(by=["id", "d_n"])
alldata_full = alldata_full.reset_index(drop=True)
alldata_id = alldata_full[["id"]]
alldata_full.head(2)

Memory usage after optimization is: 2201.14 MB
Decreased by 0.0%


,item_id,dept_id,cat_id,store_id,state_id,d_n,sales,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price,id
0,0,0,0,0,0,1,3,11101,1,1,2011,30,4,4,2,0,2.0,FOODS_1_001_CA_1_validation
1,0,0,0,0,0,2,0,11101,2,1,2011,30,4,4,2,0,2.0,FOODS_1_001_CA_1_validation


#### Separate data 

Next, separate the total data into x and y.

  **x:** columns `item_id`,	`dept_id`,	`cat_id`,	`store_id`,`state_id`,	`d_n`,	`wm_yr_wk`,	`wday`,	`month`,	`year`,	`event_name_1`,	`event_type_1`,	`event_name_2`,	`event_type_2`,	`snap` and	`sell_price`.

  **y:** columns `sales`

In [0]:
alldata_x = alldata_full.drop(columns=["sales","id"])
alldata_y = alldata_full[["sales"]]
del alldata_full

There are 31 days that are used as the testing data while the remaining is the training data.

In [0]:
pred_ind = alldata_x.index[(alldata_x["d_n"]>1913)].tolist()
train_ind = alldata_x.index[(alldata_x["d_n"]<=1913-31)].tolist()
test_ind = alldata_x.index[(alldata_x["d_n"]<=1913)&(alldata_x["d_n"]>1913-31)].tolist()

#### Normalize data

Normalization is a technique often applied as part of data preparation for machine learning. The goal of normalization is to change the values of numeric columns in the dataset to use a common scale, without distorting differences in the ranges of values. 

By observing our data, features have different ranges or scales, which indicates that normalization is required in this case.

Normalizes our data using the min/max scaler with minimum and maximum values of 0 and 1 respectively.

In [0]:
n_alldata_x = preprocessing.normalize(alldata_x)
df_n_alldata_x = pd.DataFrame(n_alldata_x)
del n_alldata_x

df_n_alldata_x = reduce_mem_usage(df_n_alldata_x)
df_n_alldata_x.columns = alldata_x.columns
del alldata_x

df_n_alldata_x.head(2)

Memory usage after optimization is: 1806.06 MB
Decreased by 75.0%


,item_id,dept_id,cat_id,store_id,state_id,d_n,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price
0,0.0,0.0,0.0,0.0,0.0,0.000089,0.983887,0.000089,0.000089,0.178223,0.002659,0.000355,0.000355,0.000177,0.0,0.000177
1,0.0,0.0,0.0,0.0,0.0,0.000177,0.983887,0.000177,0.000089,0.178223,0.002659,0.000355,0.000355,0.000177,0.0,0.000177


#### Divide data

For the normalized data, it is randomly divided into train set, test set and pred set by the seed `586`.

In [0]:
X_test = df_n_alldata_x.iloc[test_ind]
X_pred = df_n_alldata_x.iloc[pred_ind]
Y_test = alldata_y.iloc[test_ind]

n_sample_id = alldata_id.iloc[pred_ind]

In [0]:
np.random.seed(586)

ind1 = np.random.choice(train_ind, 11476436, replace=False)
ind2 = np.random.choice(train_ind, 11476436, replace=False)
ind3 = np.random.choice(train_ind, 11476436, replace=False)
ind4 = np.random.choice(train_ind, 11476436, replace=False)
ind5 = np.random.choice(train_ind, 11476436, replace=False)

In [0]:
X_train1 = df_n_alldata_x.iloc[ind1]
X_train2 = df_n_alldata_x.iloc[ind2]
X_train3 = df_n_alldata_x.iloc[ind3]
X_train4 = df_n_alldata_x.iloc[ind4]
X_train5 = df_n_alldata_x.iloc[ind5]

Y_train1 = alldata_y.iloc[ind1]
Y_train2 = alldata_y.iloc[ind2]
Y_train3 = alldata_y.iloc[ind3]
Y_train4 = alldata_y.iloc[ind4]
Y_train5 = alldata_y.iloc[ind5]

del ind1, ind2, ind3, ind4, ind5, alldata_y

#### Process data

We apply a function named `xprocess` to transfer dataframe into dictionary.

In [0]:
def xprocess(dataf):
    temps = {"all": dataf.to_numpy()}
    for i in range(len(dataf.columns)):
        temps.update({dataf.columns[i] : dataf[[dataf.columns[i]]].to_numpy()})
    return temps

In [0]:
x_test = xprocess(X_test)
x_pred = xprocess(X_pred)
del X_test, X_pred

In [0]:
x_train1 = xprocess(X_train1)
x_train2 = xprocess(X_train2)
x_train3 = xprocess(X_train3)
x_train4 = xprocess(X_train4)
x_train5 = xprocess(X_train5)
del xprocess

### RNN Model

As one class of artificial neural networks, the recurrent neural network (RNN) has connections between nodes form a directed graph along a temporal sequence, which is allowed to exhibit temporal dynamic behavior and to use their internal state (memory) to process variable length sequences of inputs.

#### Build Model

While building the RNN model, firstly, we dense input and embedding input. Then, we combine dense and embedding parts. Next, all embedding layers are flattened and all flattened layers are concatenated with inputs. At last, dense layers are added to form the final RNN model.

In [0]:
def build_model():
    tf.random.set_seed(586)
    
    all_in = tf.keras.layers.Input(shape=(16,), name="all")
    item_id_in = tf.keras.layers.Input(shape=(1,), name="item_id")
    dept_id_in = tf.keras.layers.Input(shape=(1,), name="dept_id")
    cat_id_in = tf.keras.layers.Input(shape=(1,), name="cat_id")
    store_id_in = tf.keras.layers.Input(shape=(1,), name="store_id")
    d_n_in = tf.keras.layers.Input(shape=(1,), name="d_n")
    wm_yr_wk_in = tf.keras.layers.Input(shape=(1,), name="wm_yr_wk")
    wday_in = tf.keras.layers.Input(shape=(1,), name="wday")
    month_in = tf.keras.layers.Input(shape=(1,), name="month")
    year_in = tf.keras.layers.Input(shape=(1,), name="year")
    event_name_1_in = tf.keras.layers.Input(shape=(1,), name="event_name_1")
    event_type_1_in = tf.keras.layers.Input(shape=(1,), name="event_type_1")
    event_name_2_in = tf.keras.layers.Input(shape=(1,), name="event_name_2")
    event_type_2_in = tf.keras.layers.Input(shape=(1,), name="event_type_2")
    snap_in = tf.keras.layers.Input(shape=(1,), name="snap")
    sell_price_in = tf.keras.layers.Input(shape=(1,), name="sell_price")
    
    item_id_em = tf.keras.layers.Embedding(1,1)(item_id_in)
    dept_id_em = tf.keras.layers.Embedding(1,1)(dept_id_in)
    cat_id_em = tf.keras.layers.Embedding(1,1)(cat_id_in)
    store_id_em = tf.keras.layers.Embedding(1,1)(store_id_in)
    d_n_em = tf.keras.layers.Embedding(1,1)(d_n_in)
    wm_yr_wk_em = tf.keras.layers.Embedding(1,1)(wm_yr_wk_in)
    wday_em = tf.keras.layers.Embedding(1,1)(wday_in)
    month_em = tf.keras.layers.Embedding(1,1)(month_in)
    year_em = tf.keras.layers.Embedding(1,1)(year_in)
    event_name_1_em = tf.keras.layers.Embedding(1,1)(event_name_1_in)
    event_type_1_em = tf.keras.layers.Embedding(1,1)(event_type_1_in)
    event_name_2_em = tf.keras.layers.Embedding(1,1)(event_name_2_in)
    event_type_2_em = tf.keras.layers.Embedding(1,1)(event_type_2_in)
    snap_em = tf.keras.layers.Embedding(1,1)(snap_in)
    sell_price_em = tf.keras.layers.Embedding(1,1)(sell_price_in)
    
    item_id_fl = tf.keras.layers.Flatten()(item_id_em)
    dept_id_fl = tf.keras.layers.Flatten()(dept_id_em)
    cat_id_fl = tf.keras.layers.Flatten()(cat_id_em)
    store_id_fl = tf.keras.layers.Flatten()(store_id_em)
    d_n_fl = tf.keras.layers.Flatten()(d_n_em)
    wm_yr_wk_fl = tf.keras.layers.Flatten()(wm_yr_wk_em)
    wday_fl = tf.keras.layers.Flatten()(wday_em)
    month_fl = tf.keras.layers.Flatten()(month_em)
    year_fl = tf.keras.layers.Flatten()(year_em)
    event_name_1_fl = tf.keras.layers.Flatten()(event_name_1_em)
    event_type_1_fl = tf.keras.layers.Flatten()(event_type_1_em)
    event_name_2_fl = tf.keras.layers.Flatten()(event_name_2_em)
    event_type_2_fl = tf.keras.layers.Flatten()(event_type_2_em)
    snap_fl = tf.keras.layers.Flatten()(snap_em)
    sell_price_fl = tf.keras.layers.Flatten()(sell_price_em)
    
    model = tf.keras.layers.concatenate([
        all_in,item_id_in,dept_id_in,cat_id_in,store_id_in,d_n_in,wm_yr_wk_in,wday_in,
        month_in,year_in,event_name_1_in,event_type_1_in,event_name_2_in,event_type_2_in,
        snap_in,sell_price_in,
        item_id_fl,dept_id_fl,cat_id_fl,store_id_fl,d_n_fl,wm_yr_wk_fl,wday_fl,
        month_fl,year_fl,event_name_1_fl,event_type_1_fl,event_name_2_fl,event_type_2_fl,
        snap_fl,sell_price_fl,
        ])
    
    model = tf.keras.layers.Dense(256, activation="relu")(model)
    model = tf.keras.layers.Dense(128, activation="relu")(model)
    model = tf.keras.layers.Dense(64, activation="relu")(model)
    model = tf.keras.layers.Dense(32, activation="relu")(model)
    
    denselayer = tf.keras.layers.Dense(1, activation="relu")(model)
    
    rnn_model = tf.keras.models.Model({
        "all":all_in,
        "item_id":item_id_in,
        "dept_id":dept_id_in,
        "cat_id":cat_id_in,
        "store_id":store_id_in,
        "d_n":d_n_in,
        "wm_yr_wk":wm_yr_wk_in,
        "wday":wday_in,
        "month":month_in,
        "year":year_in,
        "event_name_1":event_name_1_in,
        "event_type_1":event_type_1_in,
        "event_name_2":event_name_2_in,
        "event_type_2":event_type_2_in,
        "snap":snap_in,
        "sell_price":sell_price_in,
        }, denselayer)
        
    return rnn_model

In [0]:
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
item_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dept_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
cat_id (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
store_id (InputLayer)           [(None, 1)]          0                                            
______________________________________________________________________________________________

After applying different optimizers with different learning rate and comparing all accuracies, we choose the optimizer `Adam` and the learning rate `0.005`. Moreover, the mean_squared_error works as the loss.

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), 
                  loss=tf.keras.losses.mean_squared_error)

#### Train model

`batch_size`: Number of samples per gradient update. 

`epochs`: Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided.

Since the size of this dataset is too large, and batch_size is not work here, to deal with this situation, we seperate training data into 5 parts and call `fit` function to train the model with epochs=1 for each part of training data.

In [0]:
BATCH_SIZE = 30490
EPOCHS = 1

model.fit(x_train1, Y_train1, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test,Y_test))
model.fit(x_train2, Y_train2, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test,Y_test))
model.fit(x_train3, Y_train3, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test,Y_test))
model.fit(x_train4, Y_train4, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test,Y_test))
model.fit(x_train5, Y_train5, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test,Y_test))

Train on 11476436 samples, validate on 945190 samples
11476436/11476436 [==============================] - 37s 3us/sample - loss: 14.7613 - val_loss: 13.1010
Train on 11476436 samples, validate on 945190 samples
11476436/11476436 [==============================] - 36s 3us/sample - loss: 14.6904 - val_loss: 13.1506
Train on 11476436 samples, validate on 945190 samples
11476436/11476436 [==============================] - 39s 3us/sample - loss: 14.5038 - val_loss: 13.0287
Train on 11476436 samples, validate on 945190 samples
11476436/11476436 [==============================] - 42s 4us/sample - loss: 14.4413 - val_loss: 12.8694
Train on 11476436 samples, validate on 945190 samples
11476436/11476436 [==============================] - 40s 3us/sample - loss: 14.9703 - val_loss: 12.8244


In [0]:
del x_train1, Y_train1, x_train2, Y_train2, x_train3, Y_train3
del x_train4, Y_train4, x_train5, Y_train5, x_test, Y_test

### Prediction

The final step is to apply the RNN model on forecasting daily sales for the next 28 days (from `d1914` to `d1941`).

In [0]:
predictions = model.predict(x_pred)
del model, build_model

In [0]:
pred = pd.DataFrame(predictions)
pred.columns = ["pred"]
n_sample_id = n_sample_id.reset_index(drop=True)
pred = pd.concat([n_sample_id, pred], axis=1)
pred["f"] = np.tile(np.array(range(1,29)), 30490)
pred["f"] = pred["f"].map(lambda x: "F"+str(x))
del predictions

In [0]:
sample_submission = pd.read_csv("m5-forecasting-accuracy/sample_submission.csv")
sample_submission_evaluation = sample_submission[sample_submission["id"].str.contains("evaluation")]
del sample_submission

pred_pivot = pred.pivot(index="id", columns="f", values="pred").reset_index()
del pred

pred_pivot["id"] = pred_pivot["id"].map(lambda x: x[:-10]+"evaluation")
pred_pivot = pred_pivot.reindex(columns=sample_submission_evaluation.columns)
pred_pivot = reduce_mem_usage(pred_pivot)
pred_final = pd.merge(sample_submission_evaluation[["id"]], pred_pivot, how="left", left_on=["id"], right_on=["id"])
del pred_pivot 

Memory usage after optimization is: 1.86 MB
Decreased by 46.7%


For the forecasting table, `id` refers to a particular item that would be sold in the store mentioned. Values from `F1` to `F28` are the predicted numbers of sales in the 28-days period.

In [0]:
pred_final.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.438965,0.405029,0.383789,0.363770,0.344482,0.489258,0.504395,0.498047,0.459961,...,0.341553,0.524414,0.483887,0.445801,0.408936,0.380859,0.360107,0.340820,0.527344,0.486816
1,HOBBIES_1_002_CA_1_evaluation,1.418945,1.284180,1.151367,1.030273,0.907715,1.579102,1.624023,1.614258,1.480469,...,0.922363,1.708008,1.575195,1.440430,1.307617,1.173828,1.048828,0.928223,1.714844,1.582031
2,HOBBIES_1_003_CA_1_evaluation,1.781250,1.635742,1.500000,1.367188,1.232422,1.951172,1.998047,1.990234,1.844727,...,1.247070,2.097656,1.950195,1.802734,1.660156,1.523438,1.388672,1.255859,2.103516,1.956055
3,HOBBIES_1_004_CA_1_evaluation,1.158203,1.029297,0.908203,0.800293,0.704590,1.325195,1.370117,1.359375,1.226562,...,0.707520,1.447266,1.314453,1.179688,1.049805,0.936035,0.818359,0.717285,1.460938,1.328125
4,HOBBIES_1_005_CA_1_evaluation,1.788086,1.644531,1.509766,1.376953,1.242188,1.958008,2.003906,1.997070,1.851562,...,1.257812,2.103516,1.956055,1.808594,1.667969,1.532227,1.397461,1.264648,2.109375,1.962891


As the number of sales of each item must be an integer, so we round all predicted values from decimals into integers.

In [0]:
pred_final=pred_final.round(0)
pred_final.head(10)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,HOBBIES_1_002_CA_1_evaluation,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
2,HOBBIES_1_003_CA_1_evaluation,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0
3,HOBBIES_1_004_CA_1_evaluation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,HOBBIES_1_005_CA_1_evaluation,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0
5,HOBBIES_1_006_CA_1_evaluation,3.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,...,2.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,3.0,3.0
6,HOBBIES_1_007_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7,HOBBIES_1_008_CA_1_evaluation,3.0,3.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,...,2.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0
8,HOBBIES_1_009_CA_1_evaluation,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0
9,HOBBIES_1_010_CA_1_evaluation,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0


The prediction table is saved as a csv file for submission.

In [0]:
pred_final.to_csv("sample_submission_evaluation.csv", index=None)

### Conclusion

**Recurrent Neural Networks**, or RNNs, were designed to work with sequence prediction problems, which can model sequence of data so that each sample can be assumed to be dependent on previous ones.

In this mini project 2, we construct a RNN model on forecasting the 28-days expected sales of product groups in the USA by Walmart. 

After loading the 4 datasets, there are 4 main steps to prepare the original robust dataset into the dataset for further modelling. Firstly, we clean and reconstruct 4 datasets, including removing unnecessary columns, adding more importance message and reshaping the structures. Then, we merge separate datasets together into get a larger data frame that contains more information. Next, we normalize data into a common scale to prevent the influence of differences in the ranges of values. Last, we process the data from data frame into dictionary.

For prediction, we construct a RNN model. Since many of the features are categorical, using embedding layers is a better choice to deal with categoric inputs for neural nets by skipping the step of making dummy variables by hand.

RNNs recurrently perform the same task for every element of a sequence, with the output being depended on the previous computations. Hidden layer in RNN’s have same weights and bias through out the process giving them the chance to memorize information processed through them. The "memory" of RNNs captures information about what has been calculated so far.